In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tensorflow import keras

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
                     

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras import layers
from tensorflow.keras.models import Model
def embedds(wordtoidx,vocab_size):
        embeddings_index = {} 
        f = open('../input/glove6b/glove.6B.300d.txt', encoding="utf-8")
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
        embedding_dim = 300
        embedding_matrix = np.zeros((vocab_size, embedding_dim))
        for word, i in wordtoidx.items():
                embedding_vector = embeddings_index.get(word)
                if embedding_vector is not None:
                    embedding_matrix[i] = embedding_vector
        return  embedding_matrix  

In [ ]:
pd_train=pd.read_csv("../input/feedback-prize-effectiveness/train.csv")
text=pd_train['discourse_text']
len(text)


In [ ]:
import string
from textblob import TextBlob
from gensim.parsing.preprocessing import STOPWORDS
all_stopwords_gensim = STOPWORDS.union(set(['the', 'The']))
table = str.maketrans('', '', string.punctuation)
bad_chars = [';', ':', '!', "*", " ",",","'"]
MAX_LEN=0
import re
def correct_sentence_spelling(sentence):
    
    sentence = TextBlob(sentence)
    
    result = sentence.correct()
    
    print(result)
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase
def Data_preprocessing(text):
    list_cleaned=[]
    for i in range(len(text)):
        words =text[i].translate(table)
        #removing punctuation from our data
        words=words.split()
        stripped_filtered = [w.lower() for w in words if not w in all_stopwords_gensim]
        stripped_filtered_2=[w for w in stripped_filtered ]
        list_cleaned.append(stripped_filtered_2)
    return list_cleaned


In [ ]:
# calculate the length of the description with the most words
def max_length(descriptions):
    return max([len(d) for d in descriptions])
def create_tokenizer(descriptions):
        tokenizer = Tokenizer()
        tokenizer.fit_on_texts(descriptions)
        return tokenizer

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

def create_sequences(tokenizer, max_length, desc_list, vocab_size):
        X1=list()
        # walk through each description for the image
        for l in range(len(desc_list)):
            # encode the sequence
            seq = tokenizer.texts_to_sequences([desc_list[l]])
            # split one sequence into multiple X,y pairs
            
            in_seq = pad_sequences(seq[0:], maxlen=max_length)
            #store
            X1.append(in_seq)
        return np.array(X1)

In [ ]:
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
text_type_output = encoder.fit_transform(pd_train['discourse_type'])
text_eff_output = encoder.fit_transform(pd_train['discourse_effectiveness'])
print(text_eff_output.shape,text_type_output.shape)
desc_list=Data_preprocessing(text)
tokenizer = create_tokenizer(desc_list)
wordtoidx=tokenizer.word_index
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size,'samples Size: %d' % len(desc_list))
max_length=max_length(desc_list)


In [ ]:

inputs=create_sequences(tokenizer, max_length, desc_list, vocab_size)

print(inputs[1],desc_list[1])


In [ ]:
#now we gonna create the embedding matrx
embedding_matrx=embedds(wordtoidx,vocab_size)


In [ ]:
max_length

In [ ]:
from tensorflow.keras.layers import LeakyReLU
import tensorflow as tf
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
with tpu_strategy.scope():
    embedding_layer=layers.Embedding(vocab_size,300,mask_zero=True,weights=[embedding_matrx], input_length=360,trainable=False)
    input_layer=layers.Input(shape=(1,360))
    inputs_flattened=layers.Flatten()(input_layer)
    input_layer2=layers.Input(shape=(7,))
    input_layer2_2=layers.Dense(240)(input_layer2)
    l_relu=LeakyReLU()(input_layer2_2)
    embeds=embedding_layer(inputs_flattened)
    lstm_layer=layers.LSTM(240,return_sequences=True)(embeds)
    drop_out=layers.Dropout(0.1)(lstm_layer)
    gru_layer=layers.GRU(240)(drop_out)
    dnn_layer=layers.Dense(240,'relu')(gru_layer)
    drop_out2=layers.Dropout(0.1)(dnn_layer)
    add_layer=layers.Add()([drop_out2,l_relu])
    output1=layers.Dense(3,'softmax')(add_layer)
    model_fb=Model([input_layer,input_layer2],output1)
    model_fb.compile( optimizer='adam', loss='categorical_crossentropy',metrics='acc')
    
print(model_fb.summary())

In [ ]:
BATCH_SIZE = 16 * tpu_strategy.num_replicas_in_sync
model_history=model_fb.fit([inputs,text_type_output],text_eff_output,validation_split=0.01,epochs=20, batch_size=BATCH_SIZE)

In [ ]:
dict_types=dict()
for x in range(len(pd_train['discourse_type'])):
    dict_types[pd_train['discourse_type'][x]]=text_type_output[x]
model_fb.save('feedbacks.h5')
import pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    


#   ***preprocessing on test data***

In [ ]:

pd_test=pd.read_csv("../input/feedback-prize-effectiveness/test.csv")
text_test=pd_test['discourse_text']
desc_list_test=Data_preprocessing(text_test)
inputs_test=create_sequences(tokenizer, max_length, desc_list_test, vocab_size)
text_test

In [ ]:
inputs_test.shape

In [ ]:
inputs2=[]
for i in pd_test['discourse_type']:
    inputs2.append(dict_types[i])
    
inputs2=np.array(inputs2)

In [ ]:
inputs2

In [ ]:
predicts=model_fb.predict([inputs_test,inputs2])
print(predicts)

In [ ]:
predicts=pd.DataFrame(predicts)
predicts.rename({0: 'Adequate',1: 'Effective',2:'Ineffective'},axis='columns',inplace=True)
predicts

In [ ]:
predicts.to_csv('submission.csv', index=False)